# IMPORTING

In [2]:
from tensorflow.keras.layers import Input, Dense

class AutoEncoder(Model):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            Input(shape=(input_dim,)),
            Dense(encoding_dim, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            Input(shape=(encoding_dim,)),
            Dense(input_dim, activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM

In [37]:
# Load datasets
logon_df = pd.read_csv("../../autonomous-cybersecurity/datasets/r1/logon.csv", nrows=10000)
device_df = pd.read_csv('../../autonomous-cybersecurity/datasets/r1/device.csv', nrows=10000)
http_df = pd.read_csv('../../autonomous-cybersecurity/datasets/r1/http.csv', nrows=10000)

In [38]:
logon_df.rename(columns={"activity":"activity_logon"}, inplace=True)

In [39]:
device_df.rename(columns={"activity":"activity_device"}, inplace=True)

In [40]:
logon_df.columns, device_df.columns

(Index(['id', 'date', 'user', 'pc', 'activity_logon'], dtype='object'),
 Index(['id', 'date', 'user', 'pc', 'activity_device'], dtype='object'))

In [27]:
device_df['activity_device']

0          Connect
1          Connect
2          Connect
3          Connect
4       Disconnect
           ...    
9995       Connect
9996       Connect
9997       Connect
9998    Disconnect
9999       Connect
Name: activity_device, Length: 10000, dtype: object

In [29]:
# Preprocess data
def preprocess_data(logon_df, device_df, http_df):
    # Merge datasets on date and user
    merged_df = pd.merge(logon_df, device_df, on=['date', 'user'])
    merged_df = pd.merge(merged_df, http_df, on=['date', 'user'])

    # One-hot encoding for categorical features
    encoder = OneHotEncoder()
    encoded_pc = encoder.fit_transform(merged_df[['pc']])
    encoded_activity_logon = encoder.fit_transform(merged_df['activity_logon'])
    encoded_activity_device = encoder.fit_transform(merged_df['activity_device'])
    encoded_url = encoder.fit_transform(merged_df[['url']])

    # Create unified feature set
    features = np.concatenate((encoded_pc, encoded_activity_logon, encoded_activity_device, encoded_url), axis=1)

    return features

In [31]:
# features = preprocess_data(logon_df, device_df, http_df)

In [32]:
encoder = OneHotEncoder()

In [56]:
device_activity = encoder.fit_transform(np.array(device_df['activity_device']).reshape(-1, 1))

In [57]:
device_activity.toarray()[0]

array([1., 0.])

In [34]:
merged_df.head()

,id_x,date,user,pc_x,activity_logon,id_y,pc_y,activity_device,id,pc,url
